In [1]:
import pandas as pd
import numpy as np
from datetime import date, time, datetime, timedelta

In [9]:
from biblioteca_qca import tratamento_prazos_diarios_agendados

In [27]:
tratamento_prazos_diarios('base1302_agendados.xlsx', 'centro_de_custo.xlsx')

In [2]:
hoje = date.today()

In [3]:
df = pd.read_excel('base1402_agendados.xlsx', engine='openpyxl')

c:\Users\victoramarante\AppData\Local\Programs\Python\Python311\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [4]:
df.head()

,ID Processo,ID Prazo,NPC,Data Cadastro Prazo,Data do Prazo,Hora do Prazo,Tipo de Atividade,Sub Tipo Compromisso,Parte Cliente,Parte Adversa,...,Tipo,Subtipo do serviço,Data da contratação,Hora da contratação,Status do serviço,Status Sugestão alteração/cancelamento,Vinculado ao Recurso?,ID Recurso,Projeto,Link de Acesso à Audiência
0,510525,5328672,NaN,2023-02-13,2023-02-15,00:00:00,Prazo,Memoriais,IPIRANGA PRODUTOS DE PETROLEO,ANTONIO MADSON AREIAS DA SILVA,...,NaN,NaN,NaT,NaN,NaN,NaN,Nao,0,NaN,NaN
1,843885,5332972,1154474,2023-02-13,2023-05-25,11:10:00,Prazo,Contestação - Com Audiência,VIA VAREJO ONLINE,ADELMO AVELINO PEREIRA,...,NaN,NaN,NaT,NaN,NaN,NaN,Nao,0,NaN,NaN
2,843659,5329501,1154157,2023-02-13,2023-03-27,16:00:00,Prazo,Contestação - Com Audiência,Via Varejo S/A,ADAILTON SALES DE OLIVEIRA,...,NaN,NaN,NaT,NaN,NaN,NaN,Nao,0,NaN,NaN
3,843665,5329623,1154319,2023-02-13,2023-05-22,08:50:00,Prazo,Contestação - Com Audiência,Via Varejo S/A,GEDEVAL RAMOS DE JESUS FILHO,...,NaN,NaN,NaT,NaN,NaN,NaN,Nao,0,NaN,NaN
4,843441,5327649,1154155,2023-02-13,2023-03-17,10:10:00,Prazo,Contestação - Com Audiência,Via Varejo S/A,ELICLEIA BARBOSA DOS SANTOS,...,NaN,NaN,NaT,NaN,NaN,NaN,Nao,0,NaN,NaN


In [5]:
df.dtypes

ID Processo                                        int64
ID Prazo                                           int64
NPC                                               object
Data Cadastro Prazo                       datetime64[ns]
Data do Prazo                             datetime64[ns]
                                               ...      
Status Sugestão alteração/cancelamento            object
Vinculado ao Recurso?                             object
ID Recurso                                         int64
Projeto                                           object
Link de Acesso à Audiência                        object
Length: 66, dtype: object

In [6]:
df.shape

(2111, 66)

In [7]:
# dropando colunas que nao serao usadas
df = df.drop(['NPC', 'Número do Órgão', 'Sistema Acompanhamento', 'Responsável Cancelamento',
                          'ID Obrigação', 'Observação ', 'Observação da conclusão', 'Projeto',
                          'Link de Acesso à Audiência'],
                          axis=1)

In [8]:
# criando duas novas colunas vazias
df[' '] = ' '
df[' '] = ' '

In [9]:
df.shape

(2111, 58)

In [10]:
# selecionar o tipo de atividade = diligencia e nao selecionar os protocolos e apagar todas as linhas
df_remove = df[(df['Tipo de Atividade'] == 'Diligência') &
    (df['Sub Tipo Compromisso'].str.contains('Protocolo') == False)]

In [11]:
df = df.drop(df_remove.index)

In [12]:
centro_custo = pd.read_excel('centro_de_custo.xlsx', engine='openpyxl')

In [13]:
df_ajustado = pd.merge(df, centro_custo, on='Célula', how='left')
df_ajustado.drop(['Centro de Custo_x', 'Gestor', 'Diretoria', 'Escritorio'], axis=1, inplace=True)

df_ajustado = df_ajustado[['ID Processo', 'ID Prazo', 'Data Cadastro Prazo', 'Data do Prazo','Hora do Prazo', 'Tipo de Atividade', 'Sub Tipo Compromisso','Parte Cliente', 'Parte Adversa', 'Número do Processo', 'Órgão',
       'Comarca', 'UF', 'Tipo de Processo', 'Sistema Eletrônico','Status do Prazo', 'Data Inicio Compromisso', 'Data Conclusão','Data Protocolo', 'Data Auditoria Protocolo', 'Prazo para Protocolo',
       'Data Prazo Automático', 'Data Revisão', 'Prazo Revisão','Data de Reprovação', 'Data Cancelamento','Motivo do cancelamento do Agendamento', 'Observação de Cancelamento',
       'Fase', 'Estratégia', 'Objeto', 'Sub-Objeto', 'Produto','Adv. Responsável Processo', 'Responsável Cadastro','Responsavel Prazo', 'Responsável Revisão', 'Responsável Conclusão',
       'Responsável Protocolo', 'Responsável Auditoria Protocolo', 'Célula','Centro de Custo_y','Cliente', 'Segmento', 'Forma Abertura', 'ID PA', 'ID Prazo Origem',
       ' ', ' ', 'Tem serviço contratado?', 'Id serviço', 'Tipo','Subtipo do serviço', 'Data da contratação', 'Hora da contratação','Status do serviço', 'Status Sugestão alteração/cancelamento',
       'Vinculado ao Recurso?', 'ID Recurso']]

In [14]:
df_ajustado.rename(columns = {'Centro de Custo_y': 'Centro de Custo'}, inplace=True)

df_final = df_ajustado[(df_ajustado['Centro de Custo'] != 'AMBEV') &
            (df_ajustado['Centro de Custo'] != 'CCB MASSIFICADO') &
            (df_ajustado['Centro de Custo'] != 'Apresentação QCA') &
            (df_ajustado['Centro de Custo'] != 'EQUIPE PARAÍBA') &
            (df_ajustado['Centro de Custo'] != 'MONGERAL') &
            (df_ajustado['Centro de Custo'] != 'MARITIMO E PORTUARIO') &
            (df_ajustado['Centro de Custo'] != ' ')]

In [15]:
# from biblioteca_qca import tratamento_prazos_diarios

# import warnings
# warnings.filterwarnings('ignore')

In [16]:
# tratamento_prazos_diarios(base_agendados='base1302_agendados.xlsx',
#                             base_centro_custo='centro_de_custo.xlsx')

In [17]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2091 entries, 0 to 2097
Data columns (total 59 columns):
 #   Column                                  Non-Null Count  Dtype         
---  ------                                  --------------  -----         
 0   ID Processo                             2091 non-null   int64         
 1   ID Prazo                                2091 non-null   int64         
 2   Data Cadastro Prazo                     2091 non-null   datetime64[ns]
 3   Data do Prazo                           2091 non-null   datetime64[ns]
 4   Hora do Prazo                           2091 non-null   object        
 5   Tipo de Atividade                       2091 non-null   object        
 6   Sub Tipo Compromisso                    2091 non-null   object        
 7   Parte Cliente                           2091 non-null   object        
 8   Parte Adversa                           2091 non-null   object        
 9   Número do Processo                      2091 non-nul

In [22]:
df_final['Data Cadastro Prazo'].dt.strftime('%d/%m/%Y')

0       13/02/2023
1       13/02/2023
2       13/02/2023
3       13/02/2023
4       13/02/2023
           ...    
2093    13/02/2023
2094    13/02/2023
2095    13/02/2023
2096    13/02/2023
2097    13/02/2023
Name: Data Cadastro Prazo, Length: 2091, dtype: object

In [28]:
datetime.strptime((df_final['Data Cadastro Prazo'].dt.strptime('%d/%m/%Y')), '%d/%m/%Y').date()

AttributeError: 'DatetimeProperties' object has no attribute 'strptime'

In [33]:
import datetime

# create a datetime object representing the current date and time
now = datetime.datetime.now()

# format the datetime object as a string with the desired date format
date_string = now.strftime("%m/%d/%Y")

# parse the string into a date object
date_obj = datetime.datetime.strptime(date_string, "%m/%d/%Y").date()

# print the date object
print(date_obj)


2023-02-16
